In [ ]:
#CS Project Web Scraper

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Store URL in variable
url = 'https://www.baseball-reference.com/teams/tgl.cgi?team=HOU&t=b&year=2017'

# Access webpage
response = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})

# Store html content in a variable
page = response.content

In [ ]:
# Feed raw HTML to BeautifulSoup package
soup = BeautifulSoup(page, 'lxml')

# Finds all the tables in HTML Code
my_table = soup.find_all('table')

# Returns the table we want
table = my_table[0]

# Parses table rows
table_rows = table.find_all('tr')

# Gets header rows
header = table.find_all('th')

# Parses data we want out of header row
data = []
header_row = [i.text for i in header[:32] if len(i.text) > 0]
for column in header_row:
    if column == '':
        header_row.remove(column)     
        
# Gets rows of data and stores in a list
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    if len(row) > 0:
        data.append(row)
    else:
        continue
         
frame = pd.DataFrame(data, columns = header_row)
print(frame)

In [24]:
# Uses selenium to emulate chrome because BS cannot pull full pages, likely due to javascript
from selenium import webdriver
import warnings
warnings.filterwarnings("ignore")

def game_data_getter(path_button):
    driver = webdriver.Chrome(executable_path = r'C:/Users/Aaron/Downloads/chromedriver/chromedriver.exe')
    url = 'https://www.baseball-reference.com' + path_button
    driver.get(url)
    html = driver.page_source
    
    game_page = BeautifulSoup(html, 'html.parser')

    all_tables = game_page.find_all('table')
    print(len(all_tables))
    
    # Returns the table we want
    table = game_page.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="post_batting_HOU_clone") 

    if path_button[7:10] == 'HOU':
        status = 'Home'
    else:
        status = 'Away'
    # Parses table rows
    table_rows = table.find_all('tr')

    # Gets header rows
    header = table.find_all('th')

    data = []
    header_row = [i.text for i in header[1:22]]
    for column in header_row:
        if column == '':
            header_row.remove(column) 
    header_row.append('Player Name')
    data.append(header_row)
    
    player_names = [i.text for i in header[22:]]
    for player in player_names:
        if player == '':
            player_names.remove(player) 


    x = -1
    for tr in table_rows:
        record = []
        td = tr.find_all('td')
        record = ([i.text for i in td])
        try:
            record.append(player_names[x])
            x += 1
        except Exception:
            continue       
            
        data.append(record)

    game_data = pd.DataFrame(data, columns = data[0])
    print(game_data)
    final_output = game_data[game_data['BA'] != '']
    final_output['Game Date'] = path_button[14:23]
    final_output['H/A'] = status
    final_output = final_output.drop([0,1])
    driver.close()
    
    return(final_output)

In [ ]:
total_frame = game_data_getter(game_links[0])
for i in game_links[1:]:
    try:
        new_frame = game_data_getter(i)
    except Exception:
        new_frame = game_data_getter(i)
    total_frame = total_frame.append(new_frame)

In [ ]:
total_frame.to_csv(r'C:\Users\Aaron\Documents\Data Science SCHOOL\baseballdata.csv')

In [16]:
# Part 2, Playoff Data
import requests
from bs4 import BeautifulSoup
import pandas as pd

playoff_url = r'https://www.baseball-reference.com/postseason/'

# Access webpage
playoff_response = requests.get(playoff_url,headers={'User-Agent': 'Mozilla/5.0'})

# Store html content in a variable
page_playoffs = playoff_response.content

# Feed raw HTML to BeautifulSoup package
playoff_soup = BeautifulSoup(page_playoffs, 'lxml')

playoff_links = []
for link in playoff_soup.findAll('a', href=True):
    playoff_links.append(link['href'])
    
astros_links = []
x = 0
for link in playoff_links:
    if 'HOU' in link and '2017' in link:
        astros_links.append(playoff_links[x-1])
    x += 1

print(astros_links)    

['/postseason/2017_WS.shtml', '/postseason/2017_ALCS.shtml', '/postseason/2017_ALDS2.shtml']


In [25]:
playoff_frame = game_data_getter(astros_links[0])
for i in astros_links[1:]:
    try:
        new_po_frame = game_data_getter(i)
    except Exception:
        new_po_frame = game_data_getter(i)
    playoff_frame = playoff_frame.append(new_po_frame)
    
print(playoff_frame)

22
               Name  Jose Altuve  Carlos Beltran#  Alex Bregman  \
0              Name  Jose Altuve  Carlos Beltran#  Alex Bregman   
1            Totals         None             None          None   
2      Brad Peacock         None             None          None   
3     Josh Reddick*         None             None          None   
4   George Springer         None             None          None   
5  Justin Verlander         None             None          None   
6            Totals         None             None          None   

   Carlos Correa  Chris Devenski  Derek Fisher*  Evan Gattis  Ken Giles  \
0  Carlos Correa  Chris Devenski  Derek Fisher*  Evan Gattis  Ken Giles   
1           None            None           None         None       None   
2           None            None           None         None       None   
3           None            None           None         None       None   
4           None            None           None         None       None   
5         

KeyError: 'BA'